In [12]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from random import *
from matplotlib import pyplot as plt
import numpy as np
import GPy
from datetime import datetime

# change here
model_class_names = [1]

# randomly sample demo_light_num rows to model our function
sample_percentage = 1;

# sample sample_time_index_num time-indexes/columns from the data (1024 columns totally)
sample_time_index_num = 96;

length = 96

m_train_set_path = "./ECG200/ECG200_TRAIN"

In [13]:
# sample_time_index_num time indexes from 1024 indexes
times_indexes = np.asarray(sorted(sample(list(range(0, length)), sample_time_index_num)))
# times_indexes = np.arange(0,1024, 1024/sample_time_index_num, dtype=int);
print("We choose the following ",sample_time_index_num, "time indexes from ",length," ones\n", times_indexes)

We choose the following  96 time indexes from  96  ones
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]


In [14]:
# class name {1,2,3}
class_names = np.loadtxt(m_train_set_path, delimiter=',', usecols=[0])
m_data = np.loadtxt(m_train_set_path, delimiter=',',usecols=range(1,length+1))
print(m_data)

# normalize magnitude
m_data -= m_data.mean(1)[:,np.newaxis]
m_data /= m_data.std(1)[:,np.newaxis]


[[ 0.50206   0.54216   0.72238  ...,  0.70501   0.71382   0.43376 ]
 [ 0.14765   0.80467   0.36777  ...,  0.036857 -1.2651   -0.20802 ]
 [ 0.31665   0.2432    0.37047  ...,  0.74885   0.81804   0.53935 ]
 ..., 
 [ 0.19768   0.45542   0.97311  ..., -0.12001   0.042423  0.34329 ]
 [ 0.1795    1.0384    1.9464   ...,  0.15105   0.19338   0.45171 ]
 [ 0.073124  0.77605   2.1813   ..., -0.066689 -0.17845  -0.25605 ]]


In [15]:
indices = [i for i,cn in enumerate(class_names) if cn in model_class_names]
print("model_class_names=",model_class_names," totally has ", len(indices), " rows");

demo_num = int(sample_percentage * len(indices))
indices = sorted(sample(indices, demo_num))
# indices = indices[0:demo_light_num]
print("We select the following ",len(indices)," rows\n",indices)

model_class_names= [1]  totally has  69  rows
We select the following  6  rows
 [1, 15, 45, 59, 60, 71]


In [16]:
kern_class = GPy.kern.Matern32(input_dim=1, variance=1.5, lengthscale=2.5, active_dims=[0], name='class')
kern_replicate = GPy.kern.Matern32(input_dim=1, variance=.1, lengthscale=2.5, active_dims=[0], name='replicate')
k_hierarchy = GPy.kern.Hierarchical(kernels=[kern_class, kern_replicate])
print(k_hierarchy)

time_index_stack = np.tile(times_indexes.reshape(-1,1),(demo_num, 1)) ;

replicate_stack = []
for r in range(1, demo_num + 1):
    replicate_stack.append(np.ones(sample_time_index_num) * r);

replicate_stack = np.asarray(replicate_stack).reshape(-1,1);

filtered_data = m_data[indices]
filtered_data = filtered_data[:,times_indexes]

X = np.hstack((time_index_stack, replicate_stack))
X = np.vstack(X)
Y = np.vstack(filtered_data.reshape(-1,1))

print(X.shape)
print(Y.shape)

X_file_path = "./ECG200/model_files/X_" + str(model_class_names) + str(sample_percentage) + ".npy"
Y_file_path = "./ECG200/model_files/Y_" + str(model_class_names) + str(sample_percentage) + ".npy"

np.save(X_file_path, X)
np.save(Y_file_path, Y)

m = GPy.models.GPRegression(X=X, Y=Y, kernel=k_hierarchy)
m.optimize_restarts(num_restarts=5);
m.optimize('bfgs', messages=1)
print(m)

save_file_path = "./ECG200/model_files/model_save" + str(model_class_names) + str(sample_percentage) + ".npy"
np.save(save_file_path, m.param_array)

  hierarchy.             |  value  |  constraints  |  priors
  class.variance         |    1.5  |      +ve      |        
  class.lengthscale      |    2.5  |      +ve      |        
  replicate.variance     |    0.1  |      +ve      |        
  replicate.lengthscale  |    2.5  |      +ve      |        
(576, 2)
(576, 1)
Optimization restart 1/5, f = 290.4575977851317
Optimization restart 2/5, f = 290.19591160770074
Optimization restart 3/5, f = 290.02418200052284
Optimization restart 4/5, f = 289.8162065016461
Optimization restart 5/5, f = 289.9726872579965
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s16  0002   3.356916e+02   2.397723e+04 
    01s18  0016   2.898136e+02   2.732459e-02 
    02s20  0030   2.898131e+02   5.953781e-03 
Runtime:     02s20
Optimization status: Converged


Name : GP regression
Objective : 289.8131467922071
Number of Parameters : 5
Number of Optimization Parameters : 5
Updates : True
Parameters:
  GP_regr